In [ ]:
from keras.layers import Input, Embedding, Convolution1D, MaxPooling1D, Flatten, Dense, Dropout, merge, concatenate
#from keras.layers import concatenate
from keras.models import Model
from keras.utils import np_utils
from keras import backend as K

import tensorflow as tf
import numpy as np
from utilities import my_callbacks
from utilities import data_helper
import optparse
import sys


np.set_printoptions(threshold=np.nan)

In [ ]:
vocab = data_helper.load_all(filelist="final_data/wsj.all")
print(vocab)

In [ ]:
print("loading entity-gird for pos and neg documents...")

X_train_1, X_train_0, E = data_helper.load_and_numberize_Egrid_with_Feats("final_data/wsj.train", 
        perm_num = 20, maxlen=2000, window_size=6, vocab_list=vocab, emb_size=100)

X_dev_1, X_dev_0, E    = data_helper.load_and_numberize_Egrid_with_Feats("final_data/wsj.dev", 
        perm_num = 20, maxlen=2000, window_size=6, E = E, vocab_list=vocab, emb_size=100)

X_test_1, X_test_0, E    = data_helper.load_and_numberize_Egrid_with_Feats("final_data/wsj.test", 
        perm_num = 20, maxlen=2000, window_size=6, E = E, vocab_list=vocab, emb_size=100)

In [ ]:
num_train = len(X_train_1)
num_dev   = len(X_dev_1)
num_test  = len(X_test_1)
#assign Y value
y_train_1 = [1] * num_train 
y_dev_1 = [1] * num_dev 
y_test_1 = [1] * num_test 

print('.....................................')
print("Num of traing pairs: " + str(num_train))
print("Num of dev pairs: " + str(num_dev))
print("Num of test pairs: " + str(num_test))
#print("Num of permutation in train: " + str(opts.p_num)) 
#print("The maximum in length for CNN: " + str(opts.maxlen))
print('.....................................')

In [ ]:
# One hot encoding of the outputs
y_train_1 = np_utils.to_categorical(y_train_1, 2)
y_dev_1 = np_utils.to_categorical(y_dev_1, 2)
y_test_1 = np_utils.to_categorical(y_test_1, 2)

#randomly shuffle the training data
np.random.seed(113)
np.random.shuffle(X_train_1)
np.random.seed(113)
np.random.shuffle(X_train_0)

In [ ]:
"""
np.random.seed(113)

E = 0.01 * np.random.uniform( -1.0, 1.0, (len(vocab), 100))
E[len(vocab)-1] = 0
#E
"""

In [ ]:
def ranking_loss(y_true, y_pred):
    pos = y_pred[:, 0]
    neg = y_pred[:, 1]
    #loss = -K.sigmoid(pos-neg) # use 
    loss = K.maximum(1.0 + neg - pos, 0.0) #if you want to use margin ranking loss
    return K.mean(loss) + 0 * y_true

In [ ]:
# first, define a CNN model for sequence of entities 
sent_input = Input(shape=(2000,), dtype='int32', name='sent_input')

# embedding layer encodes the input into sequences of 300-dimenstional vectors. 

E = np.float32(E) #E was float64 which doesn't work for tensorflow conv1d function
x = Embedding(input_dim=len(vocab), output_dim=100, weights= [E], input_length=2000)(sent_input)


# add a convolutiaon 1D layer
#x = Dropout(dropout_ratio)(x)
filter_init =  tf.keras.initializers.glorot_uniform(seed=2018) 
x = Convolution1D(filters=150, kernel_size=6, padding='valid', activation='relu', kernel_initializer=filter_init)(x)


# add max pooling layers
#x = AveragePooling1D(pool_length=pool_length)(x)
x = MaxPooling1D(pool_size=6)(x)


#x = Dropout(opts.dropout_ratio)(x)
x = Flatten()(x)


#x = Dense(hidden_size, activation='relu')(x)
x = Dropout(0.5, seed=2018)(x)


# add latent coherence score
v_init = tf.keras.initializers.glorot_uniform(seed=2018) 
out_x = Dense(1, activation=None, kernel_initializer=v_init)(x)


shared_cnn = Model(sent_input, out_x)

In [ ]:
print(shared_cnn.summary())

In [ ]:
# Inputs of pos and neg document
pos_input = Input(shape=(2000,), dtype='int32', name="pos_input")
neg_input = Input(shape=(2000,), dtype='int32', name="neg_input")


# these two models will share eveything from shared_cnn
pos_branch = shared_cnn(pos_input)
neg_branch = shared_cnn(neg_input)



concatenated = merge([pos_branch, neg_branch], mode='concat', name="coherence_out")
#concatenated = concatenate([pos_branch, neg_branch], name="coherence_out")
# output is two latent coherence score


final_model = Model([pos_input, neg_input], concatenated)


#final_model.compile(loss='ranking_loss', optimizer='adam')
final_model.compile(loss={'coherence_out': ranking_loss}, optimizer="rmsprop")

# setting callback
histories = my_callbacks.Histories()

#print(shared_cnn.summary())

In [ ]:
final_model.fit([X_train_1, X_train_0], y_train_1, validation_data=None, shuffle=False, epochs=1,
                                  verbose=0, batch_size=32, callbacks=[histories])

#print(history.history.keys())
#print(history.history['loss'])

In [ ]:
for (i, loss) in enumerate(histories.losses):
    print("Iteration ",i, ":  ",loss)

In [ ]:
y_pred = final_model.predict([X_test_1, X_test_0])        

In [ ]:
ties = 0
wins = 0
n = len(y_pred)
for i in range(0,n):
    if y_pred[i][0] > y_pred[i][1]:
        wins = wins + 1
    elif y_pred[i][0] == y_pred[i][1]:
        ties = ties + 1
#print("Perform on test set after Epoch: " + str(ep) + "...!")    
print(" -Wins: " + str(wins) + " Ties: "  + str(ties))
loss = n - (wins+ties)

recall = wins/n;
prec = wins/(wins + loss)
f1 = 2*prec*recall/(prec+recall)

print(" -Test acc: " + str(wins/n))
print(" -Test f1 : " + str(f1))